In [1]:
%env AWS_REQUEST_PAYER=requester

env: AWS_REQUEST_PAYER=requester


In [2]:
from stacLib.ard_stac_search import Astac
import rasterio as rio
from rasterio.windows import Window

import numpy as np
import tqdm.notebook as tq
import zarr

In [3]:
my_date='2020-01-01/2024-12-31'

In [4]:
stac_config = {}

stac_config['url'] = 'https://landsatlook.usgs.gov/stac-server'
stac_config['fav_collection'] = 'landsat-c2l3-dswe'
stac_config['description'] = 'Landsat Collection 2 ARD Tiles DSWE'


In [5]:
H='21'
V='10'
YEAR=2021

In [6]:
_stac = Astac(stac_config)

https://landsatlook.usgs.gov/stac-server


In [7]:
date_range_text = my_date
cloud_cover_pct_max = '100'
_assets = _stac.get_my_assets(None, H, V, date_range_text, cloud_cover_pct_max)

None 2020-01-01/2024-12-31 100
440 items found


In [8]:
_assets[0]['properties']

{'start_datetime': '2023-10-26T16:36:06.7757460Z',
 'end_datetime': '2023-10-26T16:36:30.6582900Z',
 'datetime': '2023-10-26T16:36:18.717018Z',
 'platform': 'LANDSAT_9',
 'instruments': ['OLI', 'TIRS'],
 'landsat:grid_horizontal': '21',
 'landsat:grid_vertical': '10',
 'landsat:grid_region': 'CU',
 'landsat:scene_count': 2,
 'eo:cloud_cover': 93.1232,
 'landsat:cloud_shadow_cover': 5.4688,
 'landsat:snow_ice_cover': 0.0006,
 'landsat:fill': 45.423,
 'proj:epsg': None,
 'proj:shape': [5000, 5000],
 'proj:transform': [30, 0, 584415, 0, -30, 1814805],
 'created': '2023-10-30T14:11:01.126Z',
 'updated': '2023-10-30T14:11:01.126Z'}

In [9]:

def read_tif(path: str, band: int = 1, window: Window = None):
    return rio.open(path).read(band, window=window)

def read_ls_data(item, band, window):
    return read_tif(item['assets'][band]['alternate']['s3']['href'], window=window)

In [10]:
len(_assets)

440

In [11]:
# Initialize 3d array time Y X

In [12]:
window = Window(0, 0, 256, 256)

In [13]:
big3d_ary = np.zeros(shape=(len(_assets), 256, 256), dtype=np.uint16)

In [15]:
big3d_ary.shape

(440, 256, 256)

In [16]:
%%time
p_bar=tq.tqdm(range(len(_assets)), position=0, leave=True)
myband='intsm'
for t1 in range(len(_assets)):
    item = _assets[t1]
    big3d_ary[t1] = read_ls_data(item, myband, window)
    p_bar.update(1)
    p_bar.refresh()

  0%|          | 0/440 [00:00<?, ?it/s]

CPU times: user 11 s, sys: 763 ms, total: 11.8 s
Wall time: 1min 44s


In [17]:
Y=0
X=0
Z_store = f's3://eccoe-scratch/dwse/{H}{V}_{Y}{X}.zarr_store'

In [18]:
%%time
# Save the 3D NumPy array as a Zarr array
zarr.save(Z_store, big3d_ary)

# To load the Zarr array back into a NumPy array
loaded_array = zarr.load(Z_store)

# Verify the loaded array
print(loaded_array.shape)

(440, 256, 256)
CPU times: user 878 ms, sys: 97.2 ms, total: 975 ms
Wall time: 1.51 s
